# Similarity and distance metrics

scikit-fingerprints implements multiple ways to measure **similarity** or **distance** between molecules, particularly between their fingerprints. Those similarity measures and distance metrics can be used e.g. in searching, clustering, dimentionality reduction, kNN classification, and more.

## Similarities and metrics overview

In general, we can divide those measures into two groups, depending on their input:
1. Working on molecular fingerprints (vectorized molecules). There are specific metrics for binary and count fingerprints.
2. Using molecules (RDKit ``Mol`` objects) directly.

Most functions are naturally defined as similarities - the higher, the more similar two molecules are. Every similarity also has a corresponding distance function implemented, usually equal to `1 - similarity`.

Additionally, for batch computation, e.g. pairwise similarity measurements, every metric also has a bulk function, which works on whole matrices (or lists of molecules).

### Fingerprint similarities

Similarities and distances working on fingerprint vectors can be defined for binary or count fingerprints. Most similarities, however, can only be defined for binary fingerprints. This distinction is visible in function names, e.g. `tanimoto_binary_similarity` vs `tanimoto_count_similarity`. Most similarity functions have bounded value range, typically $[0, 1]$.

There are two major groups of similarities:
- only considering "on" bits, i.e. 1s in vector representations
- including both "off" and "on" bits

For two binary vectors `x` and `y`, we can define four values, which are used to define metrics:

  - $a$ – $|x \cap y|$, the number of common "on" bits
  - $b$ – the number of positions where $x$ is 1 and $y$ is 0
  - $c$ – the number of positions where $x$ is 0 and $y$ is 1
  - $d$ – the number of positions where both are 0, the number of common "off" bits

We can also mark $|x|$ as total number of "on" bits (1s) in the $x$ vector.

[Tanimoto binary similarity](https://scikit-fingerprints.github.io/scikit-fingerprints/modules/generated/skfp.distances.tanimoto_binary_similarity.html) is the most commonly used similarity measure, defined as:

  $$
  sim(x, y) = \frac{|x \cap y|}{|x \cup y|} = \frac{|x \cap y|}{|x| + |y| - |x \cap y|} = \frac{a}{a + b + c}
  $$

[Tanimoto count similarity](https://scikit-fingerprints.github.io/scikit-fingerprints/modules/generated/skfp.distances.tanimoto_count_similarity.html) is an extension to count vectors, utilizing dot product as a measure of "common bits", and vector length instead of just sum of 1s. The larger the dot product, the more similar are two vectors. It is defined as:

  $$
  sim(x, y) = \frac{x \cdot y}{\|x\|^2 + \|y\|^2 - x \cdot y}
  $$


Both variants of Tanimoto similarity use only "on" bits. [Rogot-Goldberg similarity](https://scikit-fingerprints.github.io/scikit-fingerprints/modules/generated/skfp.distances.rogot_goldberg_binary_similarity.html) is an example of similarity that includes "off" bits information. It works only for binary vectors. It is defined as:

  $$
  sim(x, y) = \frac{a}{2 \times (2a + b + c)} + \frac{d}{2 \times (2d + b + c)}
  $$

Tanimoto similarity in both versions and Rogot-Goldberg similarity have values in range $[0, 1]$.

Let's see some code examples of Tanimoto similarity usage.

We can pass the input data as either NumPy or CSR sparse arrays

In [272]:
import numpy as np
from scipy.sparse import csr_array

vec_a = [1, 0, 0]
vec_b = [0, 1, 1]

vec_a_numpy = np.array(vec_a)
vec_b_numpy = np.array(vec_b)

vec_a_sparse = csr_array([vec_a])
vec_b_sparse = csr_array([vec_b])

The distance function will yield the same result, no matter the format

In [273]:
from skfp.distances.tanimoto import tanimoto_binary_distance, tanimoto_binary_similarity

binary_sim_numpy = tanimoto_binary_similarity(vec_a_numpy, vec_b_numpy)
binary_dist_numpy = tanimoto_binary_distance(vec_a_numpy, vec_b_numpy)

binary_sim_sparse = tanimoto_binary_similarity(vec_a_sparse, vec_b_sparse)
binary_dist_sparse = tanimoto_binary_distance(vec_a_sparse, vec_b_sparse)

All "on" bits are in different positions. We expect the Tanimoto similarity to be equal to 0.

In [274]:
print(binary_sim_numpy == binary_sim_sparse == 0)

True


Thus, the distance should be equal to 1.

In [275]:
print(binary_dist_numpy == binary_dist_sparse == 1)

True


And the distance should be equal to $1 - similarity$.

In [276]:
print(binary_dist_numpy == 1 - binary_sim_numpy)
print(binary_dist_sparse == 1 - binary_sim_sparse)

True
True


Count variants work exactly the same way as binary ones.

In [277]:
import numpy as np
from scipy.sparse import csr_array

from skfp.distances.tanimoto import tanimoto_count_distance, tanimoto_count_similarity

vec_a = [2, 3, 4, 0]
vec_b = [2, 3, 4, 2]

vec_a_numpy = np.array(vec_a)
vec_b_numpy = np.array(vec_b)

vec_a_sparse = csr_array([vec_a])
vec_b_sparse = csr_array([vec_b])

In [278]:
count_sim_numpy = tanimoto_count_similarity(vec_a_numpy, vec_b_numpy)
count_dist_numpy = tanimoto_count_distance(vec_a_numpy, vec_b_numpy)

count_sim_sparse = tanimoto_count_similarity(vec_a_sparse, vec_b_sparse)
count_dist_sparse = tanimoto_count_distance(vec_a_sparse, vec_b_sparse)

In [279]:
print(count_sim_numpy == count_sim_sparse == 0.8787878787878788)

True


In [280]:
print(count_dist_numpy == count_dist_sparse == 0.12121212121212122)

True


In [281]:
print(count_dist_numpy == 1 - count_sim_numpy)
print(count_dist_sparse == 1 - count_sim_sparse)

True
True


### Molecule similarities

Some similarities include molecule structure directly in their calculation. Those can be sometimes more flexible, as they don't lose any information during the fingerprint calculation step.

[Fraggle similarity](https://scikit-fingerprints.github.io/scikit-fingerprints/modules/generated/skfp.distances.fraggle_similarity.html) is designed to be less sensitive to small changes in the middle of molecule, compared to fingerprint-based measures. It looks more on the "overall shape similarity" of molecules. Its calculation consists of a few steps:
- fragment molecule into "interesting" substructures by acyclic and ring cuts, leaving only “large” parts of a molecule (>60%)
- compare fragments with Tversky similarity, keep only appropriately similar ones
- compure [RDKit fingerprints](https://scikit-fingerprints.github.io/scikit-fingerprints/modules/generated/skfp.fingerprints.RDKitFingerprint.html) with path length 5, compare with Tanimoto similarity
- largest Tanimoto similarity is the Fraggle similarity value

This measure is asymmetric, i.e. `sim(mol_a, mol_b)` can be potentially quite different from `sim(mol_b, mol_a)`. Its value range is $[0, 1]$.

[Maximum Common Substructure (MCS) similarity](https://scikit-fingerprints.github.io/scikit-fingerprints/modules/generated/skfp.distances.mcs_similarity.html) checks the size of the maximum common substructure (MCS) between two molecules as their structural overlap, with the formula:

$$
sim(mol_a, mol_b) = \frac{numAtoms(MCS(mol_a, mol_b))}{numAtoms(mol_a) + numAtoms(mol_b) - numAtoms(MCS(mol_a, mol_b))}
$$

It also penalizes difference in molecule sizes. Its value range is $[0, 1]$.

In this case, we use RDKit ``Mol`` objects rather than vectorized data. First, prepare the query and reference molecules.

In [282]:
from rdkit.Chem import MolFromSmiles

mol_query = MolFromSmiles("COc1cc(CN2CCC(NC(=O)c3cncc(C)c3)CC2)c(OC)c2ccccc12")
mol_ref = MolFromSmiles("COc1ccccc1")

Let's call fraggle function. The default Tversky threshold is 0.8.

In [283]:
from skfp.distances.fraggle import fraggle_distance, fraggle_similarity

fraggle_sim = fraggle_similarity(mol_query, mol_ref)
fraggle_dist = fraggle_distance(mol_query, mol_ref)

Values fall within range $[0, 1]$, so we expect the distance to be $1 - similarity$.

In [284]:
print(fraggle_sim == 0.1640625)
print(fraggle_dist == 0.8359375)
print(fraggle_dist == 1 - fraggle_sim)

True
True
True


## kNN

We aim to make SKFP fully compatible with Scikit-Learn. As such, you can use similarity metrics in your machine learning pipelines, for example, as a metric in the k-Nearest Neighbors algorithm.

Prepare the data and compute the distance using the SKFP function.

In [285]:
from sklearn.neighbors import NearestNeighbors

vec_a = np.array([[0, 1, 0, 1]])
vec_b = np.array([[0, 1, 1, 0]])
skfp_dist = tanimoto_binary_distance(vec_a[0], vec_b[0])

skfp_dist

0.6666666666666667

Now, let's compare the result with Scikit-Learn's kNN implementation utilizing the SKFP metric.

In [286]:
nn = NearestNeighbors(n_neighbors=1, metric=tanimoto_binary_distance)
nn.fit(vec_a)
distances, _ = nn.kneighbors(vec_b)

distances

array([[0.66666667]])

NearestNeighbors returns a matrix, so we'll need to extract a single value for comparison.

In [287]:
sklearn_dist = distances[0][0]
print(np.isclose(skfp_dist, sklearn_dist))

True


## Bulk variants

If you need to quickly compute similarity or distance between many vectors, you may represent them as an array and pass it to **bulk** variant of a function. In the example below, the similarity is computed between i-th rows and j-th columns of both arrays.

Keep in mind that bulk variants do not support sparse arrays, as they are not valid data type for Numba, which we use for accelerating the computations.

Bulk variants are equivalent to scikit-learn's **pairwise distances**.

In [288]:
X = np.array(
    [
        [1, 1, 1],
        [0, 0, 1],
        [1, 1, 1],
    ]
)

Y = np.array(
    [
        [1, 0, 1],
        [0, 1, 1],
        [1, 1, 1],
    ]
)

X_numpy = np.array(X)
Y_numpy = np.array(Y)

In [289]:
from skfp.distances.tanimoto import (
    bulk_tanimoto_binary_distance,
    bulk_tanimoto_binary_similarity,
)

In [290]:
bulk_binary_sim_numpy = bulk_tanimoto_binary_similarity(X_numpy, Y_numpy)
bulk_binary_dist_numpy = bulk_tanimoto_binary_distance(X_numpy, Y_numpy)

In [291]:
bulk_binary_sim_numpy

array([[0.66666667, 0.66666667, 1.        ],
       [0.5       , 0.5       , 0.33333333],
       [0.66666667, 0.66666667, 1.        ]])

In [292]:
bulk_binary_dist_numpy

array([[0.33333333, 0.33333333, 0.        ],
       [0.5       , 0.5       , 0.66666667],
       [0.33333333, 0.33333333, 0.        ]])

Let's make sure that the distance is, in fact, $1−similarity$. Subtracting the similarity matrix from a scalar may look odd, but NumPy handles broadcasting nicely.

In [293]:
print((np.isclose(bulk_binary_dist_numpy, 1 - bulk_binary_sim_numpy)).all())

True


You can also use the count Tanimoto variant.

In [294]:
X = np.array(
    [
        [1, 2, 1],
        [0, 1, 3],
        [7, 1, 5],
    ]
)

Y = np.array(
    [
        [9, 0, 3],
        [8, 4, 1],
        [1, 2, 1],
    ]
)

X_numpy = np.array(X)
Y_numpy = np.array(Y)

In [295]:
from skfp.distances.tanimoto import (
    bulk_tanimoto_count_distance,
    bulk_tanimoto_count_similarity,
)

In [296]:
bulk_count_sim_numpy = bulk_tanimoto_count_similarity(X_numpy, Y_numpy)
bulk_count_dist_numpy = bulk_tanimoto_count_distance(X_numpy, Y_numpy)

In [297]:
bulk_count_sim_numpy

array([[0.14285714, 0.24285714, 1.        ],
       [0.0989011 , 0.08333333, 0.45454545],
       [0.89655172, 0.71428571, 0.20895522]])

In [298]:
bulk_count_dist_numpy

array([[0.85714286, 0.75714286, 0.        ],
       [0.9010989 , 0.91666667, 0.54545455],
       [0.10344828, 0.28571429, 0.79104478]])

In [299]:
print((np.isclose(bulk_count_dist_numpy, 1 - bulk_count_sim_numpy)).all())

True


You can pass just one array, then the similarities will be computed between its rows.

In [300]:
X = np.array(
    [
        [1, 1, 1],
        [0, 0, 0],
        [1, 1, 1],
    ]
)

In [301]:
bulk_bin_sim_one_arr = bulk_tanimoto_binary_similarity(X)
bulk_bin_sim_one_arr

array([[1., 0., 1.],
       [0., 1., 0.],
       [1., 0., 1.]])

We expect that passing two copies of the same array will yield the same result
as passing only one copy.

In [302]:
bulk_bin_sim_two_arrs = bulk_tanimoto_binary_similarity(X, X)
bulk_bin_sim_two_arrs

array([[1., 0., 1.],
       [0., 1., 0.],
       [1., 0., 1.]])

In [303]:
print(np.allclose(bulk_bin_sim_one_arr, bulk_bin_sim_two_arrs))

True


Lastly, let's prove that bulk variant produces the same result as calling
a standard one in a nested loop. Lets use some real world data.

First of all, let's transform [smiles](https://en.wikipedia.org/wiki/Simplified_Molecular_Input_Line_Entry_System) into **fingerprints**, effectively turning them into vectors.

In [304]:
from skfp.fingerprints import ECFPFingerprint

mols_list = [
    "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O",  # Ibuprofen
    "CN1C=NC2=C1C(=O)N(C(=O)N2C)C",  # caffeine
    "c1ncccc1[C@@H]2CCCN2C",  # nicotine
    "C1CC1N2C=C(C(=O)C3=CC(=C(C=C32)N4CCNCC4)F)C(=O)O",  # Ciprofloxacin
    "CC(=O)CC(C1=CC=CC=C1)C2=C(C3=CC=CC=C3OC2=O)O",  # Warfarin
    "CC(=O)Nc1ccc(O)cc1",  # Paracetamol
    "CCC[C@@H](C(=O)C(=O)NC1CC1)NC(=O)[C@@H]2[C@H]3CCC[C@H]"  # Telaprevir
    "3CN2C(=O)[C@H](C(C)(C)C)NC(=O)[C@H](C4CCCCC4)NC(=O)c5cnccn5",  # Atorvastatin
    "O=C(O)C[C@H](O)C[C@H](O)CCn2c(c(c(c2c1ccc(F)cc1)c3ccccc3)C(=O)Nc4ccccc4)C(C)C",  # Telmisartan
    "CS(=O)(=O)CCNCc1ccc(o1)c2ccc3c(c2)c(ncn3)Nc4ccc(c(c4)Cl)OCc5cccc(c5)F",  # Lapatinib
    "O=C(N)C(C)(C)CNC(=O)[C@H](C(C)C)C[C@H](O)[C@@H](N)C[C@@H](C(C)C)Cc1cc(OCCCOC)c(OC)cc1",  # Aliskiren
    "C=12CCC=3C=C(C=C(C3[C@H](C1N=CC(=C2)Br)C4CCN(CC4)C(=O)CC5CCN(CC5)C(N)=O)Br)Cl",  # Ergotamin
    r"CN1CCN(CC1)/N=C/c2c(O)c3c5C(=O)[C@@]4(C)O/C=C/[C@H](OC)[C@@H](C)[C@@H](OC(C)=O)[C@H](C)[C@H](O)[C@H](C)[C@@H](O)[C@@H](C)\C=C\C=C(\C)C(=O)Nc2c(O)c3c(O)c(C)c5O4",  # Rinfampin
    "S(c1cc(c(O)c(c1)C(C)(C)C)C(C)(C)C)C(Sc2cc(c(O)c(c2)C(C)(C)C)C(C)(C)C)(C)C",  # Probucol
    "O([C@@H]2[C@@H](O)[C@H](O[C@H]1O[C@H](CN)[C@@H](O)[C@H](O)[C@H]1O)[C@@H](N)C[C@H]2N)[C@H]3O[C@@H]([C@@H](O)[C@H](N)[C@H]3O)CO",  # Kanamycin
]

fp = ECFPFingerprint(count=True)
fps = fp.transform(mols_list)

fps[1]

array([0, 0, 0, ..., 0, 0, 0], dtype=uint32)

Notice that the input vectors use **count** data. Let's call the appropriate method.

Calculate the similarity using nested loops.

In [305]:
manual_pairwise_sim = [
    [tanimoto_count_similarity(fps[i], fps[j]) for j in range(len(fps))]
    for i in range(len(fps))
]

And now, let's use a bulk function.

In [306]:
bulk_sim = bulk_tanimoto_count_similarity(fps, fps)

Do nested loops and the bulk function yield the same result?

In [307]:
print(np.allclose(manual_pairwise_sim, bulk_sim))

True


Let's also check the performance of both approaches.

In [308]:
%timeit -r 3 -n 10 [tanimoto_count_similarity(fps[i], fps[j]) for i in range(len(fps)) for j in range(len(fps))]

14.4 ms ± 2.03 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [309]:
%timeit -r 3 -n 10 [bulk_tanimoto_count_similarity(fps, fps)]

675 µs ± 64.3 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


Bulk variant is clearly faster.